In [ ]:
import ee
import geopandas as gpd
import folium
import os
import subprocess
from shapely.geometry import mapping, MultiPolygon
from google.colab import drive
drive.mount('/content/drive')
ee.Authenticate()
# Initialize Earth Engine
ee.Initialize(project="agrisat-463314")
#ndvi-460519
#agrisat-463314

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load your shapefile
shapefile_path = f"/content/drive/MyDrive/agriculture2.0/AOI/Copie de AOImejjat.shp"
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs(epsg="4326")
geom = gdf.geometry.union_all()

# Ensure it's a MultiPolygon even if it’s a single polygon
if geom.geom_type == 'Polygon':
    geom = MultiPolygon([geom])

# Convert to GeoJSON format
aoi_geom = mapping(geom)

# Convert to Earth Engine geometry
aoi = ee.Geometry(aoi_geom)

In [5]:
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
                  .filterBounds(aoi)\
                  .filterDate('2025-07-20', '2025-08-31')\
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',(10)))
tiles = collection.toList(collection.size())

In [6]:
n_images = collection.size().getInfo()
images = collection.toList(n_images)

# Loop and extract dates
dates = []
for i in range(n_images):
    image = ee.Image(images.get(i))
    timestamp = image.date().format('YYYY-MM-dd').getInfo()
    dates.append(timestamp)
print("Acquisition dates:", dates)

Acquisition dates: ['2025-07-24', '2025-07-29', '2025-07-31', '2025-08-08', '2025-08-13', '2025-08-20', '2025-08-28']


In [7]:
d = 6
img = ee.Image(tiles.get(d))

# Sélections de bandes
red = img.select('B4')
green = img.select('B3')
blue = img.select('B2')
nir = img.select('B8')
red_edge = img.select('B5')  # ~705 nm
r531 = img.select('B5')      # Approx. 531 nm
r570 = img.select('B6')      # Approx. 570 nm
swir = img.select('B11')     # SWIR for MNDWI

# Indices standards
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI').clip(aoi)
pri = r531.subtract(r570).divide(r531.add(r570)).rename('PRI')
ndre = nir.subtract(red_edge).divide(nir.add(red_edge)).rename('NDRE')
gci = (nir.divide(green)).subtract(1).rename('GCI')
L = 0.5
savi = nir.subtract(red).multiply(1 + L).divide(nir.add(red).add(L)).rename('SAVI')
osavi = nir.subtract(red).divide(nir.add(red).add(0.16)).rename('OSAVI')
msavi2 = nir.multiply(2).add(1).subtract(
    nir.multiply(2).add(1).pow(2).subtract(nir.subtract(red).multiply(8)).sqrt()
).divide(2).rename('MSAVI2')
mndwi = green.subtract(swir).divide(green.add(swir)).rename('MNDWI')

# Indices ajoutés
# TCARI: Transformed Chlorophyll Absorption in Reflectance Index
tcari = ee.Image(3).multiply(
    red_edge.subtract(red).subtract(
        red_edge.subtract(green).multiply(0.2).multiply(red_edge.divide(red))
    )
).rename('TCARI')

# MCARI: Modified Chlorophyll Absorption in Reflectance Index
#mcari = red_edge.subtract(red).subtract((red_edge.subtract(green)).multiply(0.2)).multiply(red_edge.divide(red)).rename('MCARI')
mcari = red_edge.subtract(red).subtract(
    red_edge.subtract(green).multiply(0.2)
).multiply(red_edge.divide(green)).rename('MCARI')

# ClGreen: Chlorophyll Green Index (alternative form of GCI)
clgreen = nir.divide(green).rename('CLGREEN')

# MVI: Modified Vegetation Index
eps = ee.Number(1e-6)  # avoid divide-by-zero
ratio = nir.subtract(red).divide(nir.add(red).add(eps))
mvi = ratio.add(0.5).sqrt()

ndmi = nir.subtract(swir).divide(nir.add(swir)).rename('NDMI')
msi = swir.divide(nir).rename('MSI')
# GDD: Exemple générique (remplace `tempImg` par une vraie image de température en °C)
# GDD = Tmoy - 10 (base 10°C pour la croissance végétale)
# tempImg = ee.Image(...)  # Remplace ceci par l’image de température
# gdd = tempImg.subtract(10).max(0).rename('GDD')

# Liste des indices à exporter
ls = [ndvi, pri, ndre, gci, savi, osavi, msavi2, mndwi, tcari, mcari, clgreen, mvi,ndmi,msi]  # gdd à ajouter si disponible
ls_str = ["NDVI", "PRI", "NDRE", "GCI", "SAVI", "OSAVI", "MSAVI2", "MNDWI", "TCARI", "MCARI", 'CLGREEN', "MVI","NDMI","MSI"]  # Ajouter "GDD" si utilisé

dir_dw = f"{dates[d]}_{shapefile_path[39:53]}"
#os.mkdir(f"./{dir_dw}")
for ind, ind_st in zip(ls, ls_str):
    ee.batch.Export.image.toDrive(
        image=ind.clip(aoi),
        description=f'{ind_st}_{dates[d]}',
        folder=dir_dw,
        #fileNamePrefix=f'{ind_st}_{dates[d]}',
        scale=10,
        maxPixels=1e13,
        fileFormat='GeoTIFF'
    ).start()

# ⚠️ A exécuter aprés le téléchargement des fichiers

In [ ]:
dir = f"./{dates[d]}_{shapefile_path[39:53]}"
tifs = [f"{dir}/{tif}" for tif in os.listdir(dir) if tif.lower().endswith('.tif')]
for tif in tifs:
    print(f"Setting NoData = 'nan' for {tif}")
    subprocess.run(['gdal_edit.py', '-a_nodata', 'nan', tif])
print("✅ All files updated with NoData = nan.")

FileNotFoundError: [Errno 2] No such file or directory: './2025-08-20_OI/Copie de AO'

# Visualisation

In [8]:
import geemap

index_palettes = {
    'NDVI': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'PRI': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green'],
    'NDRE': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'GCI': ['black', 'brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'SAVI': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'OSAVI': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'MSAVI2': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'MNDWI': ['brown', 'grey', 'white', 'lightblue', 'blue', 'darkblue'],
    'TCARI': ['red', 'orange', 'yellow', 'white', 'lightgreen', 'green'],
    'MCARI': ['red', 'orange', 'yellow', 'white', 'lightgreen', 'green'],
    'CLGREEN': ['black', 'brown', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'MVI': ['red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'MSI': ['darkgreen', 'green', 'lightgreen', 'yellow', 'orange','red'],
    'NDMI': ['red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen']
}
# Initialize the map
Map = geemap.Map()
Map.centerObject(aoi, zoom=18)

# Add each index with dynamic min/max and stretched palette
for ind, ind_st in zip(ls, ls_str):
    # Compute min/max dynamically
    stats = ind.reduceRegion(
        reducer=ee.Reducer.percentile([2, 98]),  # more robust than minMax
        geometry=aoi,
        scale=10,
        maxPixels=1e13
    )

    try:
        min_val = stats.getNumber(f"{ind_st}_p2").getInfo()
        max_val = stats.getNumber(f"{ind_st}_p98").getInfo()
    except Exception:
        print(f"⚠️ Warning: Could not compute min/max for {ind_st}, skipping layer.")
        continue

    vis_params = {
        'min': min_val,
        'max': max_val,
        'palette': index_palettes[ind_st]
    }

    Map.addLayer(ind.clip(aoi), vis_params, f'{ind_st}')
    Map.add_colorbar(vis_params, label=f'{ind_st}', layer_name=f'{ind_st}',position='bottomleft')

# Add AOI overlay
Map.addLayer(aoi, {'color': 'red'}, 'Point of Interest')

# Show the map
Map

⚠️ Warning: Could not compute min/max for MVI, skipping layer.


Map(center=[31.27902137468926, -8.514163684510722], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
rgb = img.select(['B4', 'B3', 'B2']).rename(['R', 'G', 'B'])
task = ee.batch.Export.image.toDrive(
    image=rgb,
    description='RGB_Export',
    folder='Imad/20',
    fileNamePrefix='RGB_Composite_06202020',
    region=aoi,
    scale=10,
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)
task.start()